<h1 style="color: rgb(0, 0, 119);font-size: 7rem;font-family: 'Trebuchet MS', 'Lucida Sans Unicode', 'Lucida Grande', 'Lucida Sans', Arial, sans-serif;font-weight: bolder;text-align: center;">Compression d’images GUI</h1>
<br><br><br><br>

<h1 style="color: rgb(0, 172, 172);font-size: 3rem;font-family: 'Trebuchet MS', 'Lucida Sans Unicode', 'Lucida Grande', 'Lucida Sans', Arial, sans-serif;font-weight: bolder;">>>> bibliothèques :</h1>

In [1]:
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QImage, QPixmap, QPalette, QPainter, QIcon
from PyQt5.QtPrintSupport import QPrintDialog, QPrinter
from PyQt5.QtWidgets import QLabel, QComboBox, QDialog, QSizePolicy, QScrollArea, QMessageBox, QMainWindow, QMenu, QAction, \
    qApp, QFileDialog, QPushButton, QVBoxLayout
from PyQt5 import QtCore, QtGui, QtWidgets
from PIL.ImageQt import ImageQt
from matplotlib.pylab import plt
import numpy as np
from PIL import Image
from imshowpair import imshowpair
import time
import cv2
import os

<h1 style="color: rgb(0, 172, 172);font-size: 3rem;font-family: 'Trebuchet MS', 'Lucida Sans Unicode', 'Lucida Grande', 'Lucida Sans', Arial, sans-serif;font-weight: bolder;">>>> Functions :</h1>

In [2]:
def ReadImage(path):
    img = Image.open(path)
    if img.mode == 'RGB':
        img = np.array(img)
        LC = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        img = Image.fromarray(LC,"RGB")
        c_img = img.quantize(method=0,colors=256) 
        palette = c_img.getpalette()
        a_img = np.array(c_img)
        mode = 2
        return a_img,mode,palette
    elif img.mode == 'L':
        a_img = np.array(img)
        mode = 0
        return a_img,mode
    elif img.mode == 'P':
        c_img = img.quantize(method=0,colors=256) 
        palette = c_img.getpalette()
        a_img = np.array(c_img)
        mode = 1
        return a_img,mode,palette
def img_to_txt(I):
    sh = I.shape
    im = I.flatten()
    T = ''
    for i in im:
        T += chr(int(i))
    return T,sh

def txt_to_img(T,sh):
    M = []
    for i in T:
        M.append(ord(i))
    return np.array(M).reshape(sh)
def codage_LZW(M):
    w = ""
    code = []
    dict_size = 256
    D = {chr(i): i for i in range(dict_size)}
    for c in M:
        wc = w + c
        if wc in D:
            w = wc
        else:
            code.append(D[w])
            D[wc] = dict_size
            dict_size += 1
            w = c
    if w:
        code.append(D[w])
    return code
def decodage_LZW(M):
    dict_size = 256
    D = {i: chr(i) for i in range(dict_size)}
    w = chr(M[0])
    M = M[1:]
    code = w
    e = w
    for i in M:
        if i in D.keys():
            e = D[i]
        elif i == dict_size:
            e = w + w[0]
        code += e
        D[dict_size] = w + e[0]
        dict_size += 1
        w = e
    return code
def LZW_img(I,op):
    if op == "decodage":
        if I[3] == 0:
            i_img = txt_to_img(decodage_LZW(I[6:]),I[4:6])
            c_img = Image.fromarray(i_img.astype(np.uint8),"L")
        elif I[3] == 1 :
            i_img = txt_to_img(decodage_LZW(I[774:]),I[4:6])
            c_img = Image.fromarray(i_img.astype(np.uint8),"P")
            c_img.putpalette(I[6:774].tolist())
        elif I[3] == 2 :
            i_img = txt_to_img(decodage_LZW(I[774:]),I[4:6])
            c_img = Image.fromarray(i_img.astype(np.uint8),"P")
            c_img.putpalette(I[6:774].tolist())
            c_img = c_img.convert("RGB")
            f_img = cv2.cvtColor(np.array(c_img), cv2.COLOR_YCrCb2RGB)
            c_img = Image.fromarray(f_img,"RGB")
        return c_img
    elif op == "codage":
        C = img_to_txt(I[0])
        CC = [73,  82,  77]
        if I[1] == 0:
            CC.append(I[1])
            CC.extend(C[1])
            CC.extend(codage_LZW(C[0]))
        else :
            CC.append(I[1])
            CC.extend(C[1])
            CC.extend(I[2])
            CC.extend(codage_LZW(C[0]))
        return np.array(CC)
def pil2pixmap(self, im):
        if im.mode == "RGB":
            r, g, b = im.split()
            im = Image.merge("RGB", (b, g, r))
        elif  im.mode == "RGBA":
            r, g, b, a = im.split()
            im = Image.merge("RGBA", (b, g, r, a))
        elif im.mode == "L":
            im = im.convert("RGBA")
        im2 = im.convert("RGBA")
        data = im2.tobytes("raw", "RGBA")
        qim = QImage(data, im.size[0], im.size[1], QImage.Format_ARGB32)
        pixmap = QPixmap.fromImage(qim)
        return pixmap

<h1 style="color: rgb(0, 172, 172);font-size: 3rem;font-family: 'Trebuchet MS', 'Lucida Sans Unicode', 'Lucida Grande', 'Lucida Sans', Arial, sans-serif;font-weight: bolder;">>>> Modules graphiques :</h1>

In [3]:
class Ui_Dialog(object):
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(522, 390)
        Dialog.setWindowIcon(QIcon('Logo.ico'))
        self.verticalLayoutWidget = QtWidgets.QWidget(Dialog)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(0, 20, 521, 301))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout_2 = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout_2.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.horizontalLayout = QtWidgets.QHBoxLayout()
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.label_2 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_2.setStyleSheet("font: italic 10pt \"MS Sans Serif\";\n"
"font: 10pt \"MS Shell Dlg 2\";")
        self.label_2.setAlignment(QtCore.Qt.AlignCenter)
        self.label_2.setObjectName("label_2")
        self.horizontalLayout.addWidget(self.label_2)
        self.verticalLayout_2.addLayout(self.horizontalLayout)
        self.horizontalLayout_2 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_2.setObjectName("horizontalLayout_2")
        self.label_3 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_3.setStyleSheet("font: 75 10pt \"MS Shell Dlg 2\";")
        self.label_3.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.label_3.setObjectName("label_3")
        self.horizontalLayout_2.addWidget(self.label_3)
        self.label_4 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_4.setStyleSheet("font: italic 10pt \"MS Sans Serif\";\n"
"font: 10pt \"MS Shell Dlg 2\";")
        self.label_4.setObjectName("label_4")
        self.horizontalLayout_2.addWidget(self.label_4)
        self.verticalLayout_2.addLayout(self.horizontalLayout_2)
        self.horizontalLayout_3 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_3.setObjectName("horizontalLayout_3")
        self.label_5 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_5.setStyleSheet("font: 75 10pt \"MS Shell Dlg 2\";")
        self.label_5.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.label_5.setObjectName("label_5")
        self.horizontalLayout_3.addWidget(self.label_5)
        self.label_6 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_6.setStyleSheet("font: italic 10pt \"MS Sans Serif\";\n"
"font: 10pt \"MS Shell Dlg 2\";")
        self.label_6.setObjectName("label_6")
        self.horizontalLayout_3.addWidget(self.label_6)
        self.verticalLayout_2.addLayout(self.horizontalLayout_3)
        self.horizontalLayout_5 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_5.setObjectName("horizontalLayout_5")
        self.label_9 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_9.setStyleSheet("font: 75 10pt \"MS Shell Dlg 2\";")
        self.label_9.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.label_9.setObjectName("label_9")
        self.horizontalLayout_5.addWidget(self.label_9)
        self.label_10 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_10.setStyleSheet("font: italic 10pt \"MS Sans Serif\";\n"
"font: 10pt \"MS Shell Dlg 2\";")
        self.label_10.setObjectName("label_10")
        self.horizontalLayout_5.addWidget(self.label_10)
        self.verticalLayout_2.addLayout(self.horizontalLayout_5)
        self.horizontalLayout_8 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_8.setObjectName("horizontalLayout_8")
        self.label_13 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_13.setStyleSheet("font: 75 10pt \"MS Shell Dlg 2\";")
        self.label_13.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.label_13.setObjectName("label_13")
        self.horizontalLayout_8.addWidget(self.label_13)
        self.label_14 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_14.setStyleSheet("font: italic 10pt \"MS Sans Serif\";\n"
"font: 10pt \"MS Shell Dlg 2\";")
        self.label_14.setObjectName("label_14")
        self.horizontalLayout_8.addWidget(self.label_14)
        self.verticalLayout_2.addLayout(self.horizontalLayout_8)
        self.horizontalLayout_4 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_4.setObjectName("horizontalLayout_4")
        self.label_7 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_7.setStyleSheet("font: 75 10pt \"MS Shell Dlg 2\";")
        self.label_7.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.label_7.setObjectName("label_7")
        self.horizontalLayout_4.addWidget(self.label_7)
        self.label_8 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_8.setStyleSheet("font: italic 10pt \"MS Sans Serif\";\n"
"font: 10pt \"MS Shell Dlg 2\";")
        self.label_8.setObjectName("label_8")
        self.horizontalLayout_4.addWidget(self.label_8)
        self.verticalLayout_2.addLayout(self.horizontalLayout_4)
        self.horizontalLayout_6 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_6.setObjectName("horizontalLayout_6")
        self.label_11 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_11.setStyleSheet("font: 75 10pt \"MS Shell Dlg 2\";")
        self.label_11.setAlignment(QtCore.Qt.AlignRight|QtCore.Qt.AlignTrailing|QtCore.Qt.AlignVCenter)
        self.label_11.setObjectName("label_11")
        self.horizontalLayout_6.addWidget(self.label_11)
        self.label_12 = QtWidgets.QLabel(self.verticalLayoutWidget)
        self.label_12.setStyleSheet("font: italic 10pt \"MS Sans Serif\";\n"
"font: 10pt \"MS Shell Dlg 2\";")
        self.label_12.setObjectName("label_12")
        self.horizontalLayout_6.addWidget(self.label_12)
        self.verticalLayout_2.addLayout(self.horizontalLayout_6)
        self.horizontalLayoutWidget_7 = QtWidgets.QWidget(Dialog)
        self.horizontalLayoutWidget_7.setGeometry(QtCore.QRect(0, 320, 521, 80))
        self.horizontalLayoutWidget_7.setObjectName("horizontalLayoutWidget_7")
        self.horizontalLayout_7 = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget_7)
        self.horizontalLayout_7.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout_7.setObjectName("horizontalLayout_7")
        spacerItem = QtWidgets.QSpacerItem(40, 20, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Minimum)
        self.horizontalLayout_7.addItem(spacerItem)
        self.pushButton = QtWidgets.QPushButton(self.horizontalLayoutWidget_7)
        self.pushButton.setObjectName("pushButton")
        self.horizontalLayout_7.addWidget(self.pushButton)
        spacerItem1 = QtWidgets.QSpacerItem(40, 20, QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Minimum)
        self.horizontalLayout_7.addItem(spacerItem1)

        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Dialog"))
        self.label_2.setText(_translate("Dialog", "ccc"))
        self.label_3.setText(_translate("Dialog", "Image Format : "))
        self.label_4.setText(_translate("Dialog", "ccc"))
        self.label_5.setText(_translate("Dialog", "Color Mode : "))
        self.label_6.setText(_translate("Dialog", "ccc"))
        self.label_9.setText(_translate("Dialog", "Image Resolution : "))
        self.label_10.setText(_translate("Dialog", "ccc"))
        self.label_13.setText(_translate("Dialog", "Image Size : "))
        self.label_14.setText(_translate("Dialog", "ccc"))
        self.label_7.setText(_translate("Dialog", "Image creation date : "))
        self.label_8.setText(_translate("Dialog", "ccc"))
        self.label_11.setText(_translate("Dialog", "Image modification date : "))
        self.label_12.setText(_translate("Dialog", "ccc"))
        self.pushButton.setText(_translate("Dialog", "OK"))

class Ui_Form(object):
    def setupUi(self, Form):
        Form.setObjectName("Form")
        Form.resize(567, 173)
        self.label_2 = QtWidgets.QLabel(Form)
        self.label_2.setGeometry(QtCore.QRect(270, 40, 51, 31))
        self.label_2.setStyleSheet("background : rgb(176, 176, 176);\n"
"font: 63 9pt \"Segoe UI Variable Text Semibold\";\n"
"font: 75 8pt \"MS Serif\";\n"
"font: 9pt \"MS Shell Dlg 2\";\n"
"border-radius: 25px;\n"
"")
        self.label_2.setAlignment(QtCore.Qt.AlignCenter)
        self.label_2.setObjectName("label_2")
        self.label_3 = QtWidgets.QLabel(Form)
        self.label_3.setGeometry(QtCore.QRect(360, 40, 51, 31))
        self.label_3.setStyleSheet("background : rgb(176, 176, 176);\n"
"font: 63 9pt \"Segoe UI Variable Text Semibold\";\n"
"font: 75 8pt \"MS Serif\";\n"
"font: 9pt \"MS Shell Dlg 2\";\n"
"border-radius: 25px;")
        self.label_3.setAlignment(QtCore.Qt.AlignCenter)
        self.label_3.setObjectName("label_3")
        self.label_5 = QtWidgets.QLabel(Form)
        self.label_5.setGeometry(QtCore.QRect(270, 110, 51, 31))
        self.label_5.setStyleSheet("background : rgb(176, 176, 176);\n"
"font: 63 9pt \"Segoe UI Variable Text Semibold\";\n"
"font: 75 8pt \"MS Serif\";\n"
"font: 9pt \"MS Shell Dlg 2\";\n"
"border-radius: 25px;\n"
"")
        self.label_5.setAlignment(QtCore.Qt.AlignCenter)
        self.label_5.setObjectName("label_5")
        self.label_6 = QtWidgets.QLabel(Form)
        self.label_6.setGeometry(QtCore.QRect(360, 110, 51, 31))
        self.label_6.setStyleSheet("background : rgb(176, 176, 176);\n"
"font: 63 9pt \"Segoe UI Variable Text Semibold\";\n"
"font: 75 8pt \"MS Serif\";\n"
"font: 9pt \"MS Shell Dlg 2\";\n"
"border-radius: 25px;")
        self.label_6.setAlignment(QtCore.Qt.AlignCenter)
        self.label_6.setObjectName("label_6")
        self.label_4 = QtWidgets.QLabel(Form)
        self.label_4.setGeometry(QtCore.QRect(0, 100, 489, 41))
        self.label_4.setStyleSheet("font: 63 12pt \"Segoe UI Semibold\";")
        self.label_4.setTextFormat(QtCore.Qt.PlainText)
        self.label_4.setAlignment(QtCore.Qt.AlignCenter)
        self.label_4.setObjectName("label_4")
        self.label = QtWidgets.QLabel(Form)
        self.label.setGeometry(QtCore.QRect(-2, 10, 491, 81))
        self.label.setStyleSheet("font: 63 12pt \"Segoe UI Semibold\";")
        self.label.setTextFormat(QtCore.Qt.PlainText)
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label.setObjectName("label")

        self.retranslateUi(Form)
        QtCore.QMetaObject.connectSlotsByName(Form)

    def retranslateUi(self, Form):
        _translate = QtCore.QCoreApplication.translate
        Form.setWindowTitle(_translate("Form", "Form"))
        self.label_2.setText(_translate("Form", "Ctrl"))
        self.label_3.setText(_translate("Form", "O"))
        self.label_5.setText(_translate("Form", "Ctrl"))
        self.label_6.setText(_translate("Form", "A"))
        self.label_4.setText(_translate("Form", "         About               + "))
        self.label.setText(_translate("Form", "Open Image               + "))

class Form(object):
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(392, 152)
        Dialog.setWindowIcon(QIcon('Logo.ico'))
        Dialog.setSizeGripEnabled(False)
        Dialog.setModal(False)
        self.verticalLayoutWidget = QtWidgets.QWidget(Dialog)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(30, 30, 331, 101))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")
        self.comboBox = QtWidgets.QComboBox(self.verticalLayoutWidget)
        self.comboBox.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
        self.comboBox.setObjectName("comboBox")
        self.comboBox.addItem("")
        self.comboBox.addItem("")
        self.comboBox.addItem("")
        self.comboBox.addItem("")
        self.verticalLayout.addWidget(self.comboBox)
        self.horizontalLayout = QtWidgets.QHBoxLayout()
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.pushButton = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton.setObjectName("pushButton")
        self.horizontalLayout.addWidget(self.pushButton)

        self.pushButton_2 = QtWidgets.QPushButton(self.verticalLayoutWidget)
        self.pushButton_2.setObjectName("pushButton_2")
        self.horizontalLayout.addWidget(self.pushButton_2)
        self.verticalLayout.addLayout(self.horizontalLayout)
        self.retranslateUi(Dialog)
        QtCore.QMetaObject.connectSlotsByName(Dialog)

    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Dialog"))
        self.comboBox.setItemText(0, _translate("Dialog", "IRM"))
        self.comboBox.setItemText(1, _translate("Dialog", "JPEG"))
        self.comboBox.setItemText(2, _translate("Dialog", "BMP"))
        self.comboBox.setItemText(3, _translate("Dialog", "GIF"))
        self.pushButton.setText(_translate("Dialog", "Cancel"))
        self.pushButton_2.setText(_translate("Dialog", "Save"))

<h1 style="color: rgb(0, 172, 172);font-size: 3rem;font-family: 'Trebuchet MS', 'Lucida Sans Unicode', 'Lucida Grande', 'Lucida Sans', Arial, sans-serif;font-weight: bolder;">>>> Module principale :</h1>

In [4]:
class QImageViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowIcon(QIcon('Logo.ico'))
        self.printer = QPrinter()
        self.scaleFactor = 0.0
        
        Form = QtWidgets.QWidget()
        Form.ui = Ui_Form()
        Form.ui.setupUi(Form)

        self.imageLabel = QLabel()
        self.imageLabel.setBackgroundRole(QPalette.Base)
        self.imageLabel.setSizePolicy(QSizePolicy.Ignored, QSizePolicy.Ignored)
        self.imageLabel.setScaledContents(True)

        self.scrollArea = QScrollArea()
        self.scrollArea.setBackgroundRole(QPalette.Dark)
        self.scrollArea.setWidget(Form)
        self.scrollArea.setVisible(True)
        self.scrollArea.setAlignment(Qt.AlignCenter )
        
        self.createActions()
        self.createMenus()
        
        self.setWindowTitle("Compresseur d'image IRM")
        self.resize(1000, 700)
        self.setCentralWidget(self.scrollArea)
#         self.imageLabel.setContentsMargins(10,10,10,10)

    def open(self):
        options = QFileDialog.Options()
        self.fileName, _ = QFileDialog.getOpenFileName(self, 'QFileDialog.getOpenFileName()', '',
                                                  'Images (*.png *.jpeg *.jpg *.bmp *.gif *.irm)', options=options)
        self.filextension = os.path.splitext(self.fileName)[1][1:]
        if self.fileName:
            
            if self.filextension == 'irm':
                with open(self.fileName,'rb') as op:
                    res = np.frombuffer(op.read(),dtype=int)
                self.image = LZW_img(res,"decodage")
            else : 
                self.image = Image.open(self.fileName)
    
            pixmap = pil2pixmap(self,self.image)
            if not pixmap:
                QMessageBox.information(self, "Image Viewer", "Cannot load %s." % fileName)
                return
            pixmap = pixmap.scaled(950, 750, Qt.KeepAspectRatio, Qt.FastTransformation)
            self.imageLabel.setPixmap(pixmap)
            self.scrollArea.setWidget(self.imageLabel)
            self.scaleFactor = 1.0
            
#             self.scrollArea.setWidget(self.imageLabel)
            self.scrollArea.setVisible(True)
            self.saveAct.setEnabled(True)
            self.printAct.setEnabled(True)
            self.infoAct.setEnabled(True)
            self.fitToWindowAct.setEnabled(True)
            self.updateActions()

            if not self.fitToWindowAct.isChecked():
                self.imageLabel.adjustSize()

    def save(self):
        dialog = QDialog()
        dialog.ui = Form()
        dialog.ui.setupUi(dialog)
        action1 = QAction("&Cancel111...", self)
        dialog.ui.pushButton.clicked.connect(lambda: self.Cancel_clicked(dialog))
        dialog.ui.pushButton_2.clicked.connect(lambda: self.Save_clicked(dialog))
        dialog.exec()

    def Cancel_clicked(self,dialog):
        dialog.close()
    def Save_clicked(self,dialog):
        C = dialog.ui.comboBox.currentText()
        if C == 'IRM':
            img = ReadImage(self.fileName)
            L = LZW_img(img,"codage")
            name = QFileDialog.getSaveFileName(self, 'Save Image')
            with open(name[0]+".irm",'wb') as op:
                op.write(L.tobytes())
        else : 
            img = Image.open(self.fileName)
            name = QFileDialog.getSaveFileName(self, 'Save Image')
            img.save(name[0]+"."+C.lower(), 
                 C, 
                 optimize = True, 
                 quality = 10)
        QMessageBox.about(self, "", "The image is saved successfully")
        dialog.close()
                
    def print_(self):
        dialog = QPrintDialog(self.printer, self)
        if dialog.exec_():
            painter = QPainter(self.printer)
            rect = painter.viewport()
            size = self.imageLabel.pixmap().size()
            size.scale(rect.size(), Qt.KeepAspectRatio)
            painter.setViewport(rect.x(), rect.y(), size.width(), size.height())
            painter.setWindow(self.imageLabel.pixmap().rect())
            painter.drawPixmap(0, 0, self.imageLabel.pixmap())
    
    def info(self):
        img = self.image
        ti_c = os.path.getctime(self.fileName)
        ti_m = os.path.getmtime(self.fileName)
        dialog = QDialog()
        dialog.ui = Ui_Dialog()
        dialog.ui.setupUi(dialog)
        dialog.ui.label_2.setText(self.fileName)
        dialog.ui.label_4.setText(self.filextension.upper())
        dialog.ui.label_6.setText(img.mode)
        dialog.ui.label_10.setText(str(img.size[0])+' x '+str(img.size[1]))
        dialog.ui.label_14.setText('{:.2f}'.format(os.path.getsize(self.fileName)/1024)+'Kb')
        dialog.ui.label_8.setText(time.ctime(ti_c))
        dialog.ui.label_12.setText(time.ctime(ti_m))
        dialog.ui.pushButton.clicked.connect(lambda: self.Cancel_clicked(dialog))
        dialog.exec()

    def zoomIn(self):
        self.scaleImage(1.25)

    def zoomOut(self):
        self.scaleImage(0.8)

    def normalSize(self):
        self.imageLabel.adjustSize()
        self.scaleFactor = 1.0

    def fitToWindow(self):
        fitToWindow = self.fitToWindowAct.isChecked()
        self.scrollArea.setWidgetResizable(fitToWindow)
        if not fitToWindow:
            self.normalSize()

        self.updateActions()

    def about(self):
        QMessageBox.about(self, "About IRM Image Viewer",
                          "<p>This App is an image viewer and converter desktop application.</p>"
                          "<p>This app can read and write images with file formats BMP,"
                          "JPG, GIF, and our new file format IRM.</p>")

    def createActions(self):
        self.openAct = QAction("&Open...", self, shortcut="Ctrl+O", triggered=self.open)
        self.saveAct = QAction("&Save...", self, shortcut="Ctrl+S", enabled=False, triggered=self.save)
        self.printAct = QAction("&Print...", self, shortcut="Ctrl+P", enabled=False, triggered=self.print_)
        self.exitAct = QAction("E&xit", self, shortcut="Ctrl+E", triggered=self.close)
        self.infoAct = QAction("Image &Info", self, enabled=False, shortcut="Ctrl+I", triggered=self.info)
        self.zoomInAct = QAction("Zoom &In (25%)", self, shortcut="Ctrl++", enabled=False, triggered=self.zoomIn)
        self.zoomOutAct = QAction("Zoom &Out (25%)", self, shortcut="Ctrl+-", enabled=False, triggered=self.zoomOut)
        self.normalSizeAct = QAction("&Normal Size", self, shortcut="Ctrl+N", enabled=False, triggered=self.normalSize)
        self.fitToWindowAct = QAction("&Fit to Window", self, enabled=False, checkable=True, shortcut="Ctrl+W",
                                      triggered=self.fitToWindow)
        self.aboutAct = QAction("&About", self, shortcut="Ctrl+A", triggered=self.about)
        self.aboutQtAct = QAction("About &Qt", self, triggered=qApp.aboutQt)

    def createMenus(self):
        self.fileMenu = QMenu("&File", self)
        self.fileMenu.addAction(self.openAct)
        self.fileMenu.addAction(self.saveAct)
        self.fileMenu.addAction(self.printAct)
        self.fileMenu.addSeparator()
        self.fileMenu.addAction(self.exitAct)
        
        self.infoMenu = QMenu("&Info", self)
        self.infoMenu.addAction(self.infoAct)

        self.viewMenu = QMenu("&View", self)
        self.viewMenu.addAction(self.zoomInAct)
        self.viewMenu.addAction(self.zoomOutAct)
        self.viewMenu.addAction(self.normalSizeAct)
        self.viewMenu.addSeparator()
        self.viewMenu.addAction(self.fitToWindowAct)

        self.helpMenu = QMenu("&Help", self)
        self.helpMenu.addAction(self.aboutAct)
        self.helpMenu.addAction(self.aboutQtAct)

        self.menuBar().addMenu(self.fileMenu)
        self.menuBar().addMenu(self.infoMenu)
        self.menuBar().addMenu(self.viewMenu)
        self.menuBar().addMenu(self.helpMenu)

    def updateActions(self):
        self.zoomInAct.setEnabled(not self.fitToWindowAct.isChecked())
        self.zoomOutAct.setEnabled(not self.fitToWindowAct.isChecked())
        self.normalSizeAct.setEnabled(not self.fitToWindowAct.isChecked())

    def scaleImage(self, factor):
        self.scaleFactor *= factor
        self.imageLabel.resize(self.scaleFactor * self.imageLabel.pixmap().size())

        self.adjustScrollBar(self.scrollArea.horizontalScrollBar(), factor)
        self.adjustScrollBar(self.scrollArea.verticalScrollBar(), factor)

        self.zoomInAct.setEnabled(self.scaleFactor < 4.0)
        self.zoomOutAct.setEnabled(True)

    def adjustScrollBar(self, scrollBar, factor):
        scrollBar.setValue(int(factor * scrollBar.value()
                               + ((factor - 1) * scrollBar.pageStep() / 2)))


if __name__ == '__main__':
    import sys
    from PyQt5.QtWidgets import QApplication

    app = QApplication(sys.argv)
    imageViewer = QImageViewer()
    imageViewer.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\HP\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
